## DPENet

#### Import module

In [1]:
import glob
import json
import os
import numpy as np
import math
from tqdm.notebook import trange, tqdm
from time import sleep
import torch
import torch.nn as nn
from torch import optim
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from tensorboardX import SummaryWriter
from torchsummary import summary
from matplotlib import pyplot as plt

#### Load Input and GT

GT format  
- ['ego_global_x', 'ego_global_y']  

Input format  
- ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']

In [2]:
gt_folder = './DDPE_GT_adaf_1/'
in_folder = './DDPE_input_adaf_1/'
gt_file_list = glob.glob(gt_folder+'/*_gt.json')
gt_file_list.sort()
in_train, gt_train = [], []
in_test, gt_test = [], []

In [3]:
# Load the json log.
for k in trange(len(gt_file_list),desc='folder progress'):
    # Load the GT
    file_name_gt = gt_file_list[k]
    with open(file_name_gt, 'r', encoding='utf-8-sig') as data_file_gt:
        try:
            data_gt = json.load(data_file_gt)
        except:
            continue
    
    # Load the input
    file_name_in = in_folder + gt_file_list[k][17:26] + '_in.json'
    with open(file_name_in, 'r', encoding='utf-8-sig') as data_file_in:
        try:
            data_in = json.load(data_file_in)
        except:
            continue
            
    if len(data_gt['ego_global_y']) < 60:
        continue
    if data_in['ego_vx'] == 0.0:
        continue
        
    if k % 10 == 0:
        gt_test.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_test.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])
    else:
        gt_train.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_train.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])

folder progress:   0%|          | 0/533745 [00:00<?, ?it/s]

In [4]:
# make list to tensor
gt_train = torch.tensor(gt_train,requires_grad=False).cuda()
in_train = torch.tensor(in_train,requires_grad=False).cuda()
gt_test = torch.tensor(gt_test,requires_grad=False).cuda()
in_test = torch.tensor(in_test,requires_grad=False).cuda()

In [5]:
# normalize the input dataset
# Vehicle CAN Information on SV Test/Demo Vehicles
# https://docs.stradvision.com/pages/viewpage.action?pageId=106747304
# ego_vx : 0 to 511.96875 (km/h)
# yaw_rate : -40.95 to 40.96(˚/s)
# sas_angle : -3276.8 to 3276.7 (deg)
# long_acc : -10.23 to 10.24 (m/s^2)
# lat_acc : -10.23 to 10.24 (m/s^2)

# train dataset max
# [120.7656,  24.1400, 305.8000,   4.1700,   4.9700]
# train dataset min
# [0.0000,  -31.4100, -473.7000,   -7.7000,   -5.3800]
# train dataset mean
# [57.9791, -0.1554, -0.5714,  0.0685,  0.0752]
# train datset std
# [35.1523,  2.0590, 18.6299,  0.5983,  0.4319]

# normalize
# (in - mean) / std
in_train = (in_train - torch.mean(in_train,0)) / torch.std(in_train,0)
in_test = (in_test - torch.mean(in_train,0)) / torch.std(in_train,0)
# range [0 1]
# minus min -> divide by max
in_train = in_train - torch.min(in_train,0).values
in_train = in_train / torch.max(in_train,0).values
in_test = in_test - torch.min(in_train,0).values
in_test = in_test / torch.max(in_train,0).values

# range [-1 1]
# minus min -> divide by max/2 -> minus 1


In [6]:
# class to use the dataloader
class DPEDataset(data.Dataset):
    def __len__(self):
        return len(in_train)
    
    def __getitem__(self,idx):
        in_tmp = in_train[idx]
        gt_tmp = gt_train[idx]
        return {'in':in_tmp,'gt':gt_tmp}

In [7]:
class DPETestset(data.Dataset):
    def __len__(self):
        return len(in_test)
    
    def __getitem__(self,idx):
        in_tm = in_test[idx]
        gt_tm = gt_test[idx]
        return {'in':in_tm,'gt':gt_tm}

In [8]:
dpe_dataset = DPEDataset()
dpe_testset = DPETestset()

In [9]:
len(gt_train), len(in_train), len(gt_test), len(in_test)

(425054, 425054, 47231, 47231)

In [ ]:
# Exercise

In [ ]:
print(in_train.requires_grad)

In [ ]:
in_train[0]

In [ ]:
# visualize distribution
ls_ego_vx, ls_yaw_rate, ls_sas_angle, ls_long_acc, ls_lat_acc = [], [], [], [], []
for i in range(len(in_train)):
    ls_ego_vx.append(in_train[i][0])
    ls_yaw_rate.append(in_train[i][1])
    ls_sas_angle.append(in_train[i][2])
    ls_long_acc.append(in_train[i][3])
    ls_lat_acc.append(in_train[i][4])

In [ ]:
min(ls_ego_vx), max(ls_ego_vx)

In [ ]:
# check the ego_vx distribution
ls_ego_vx_cat = np.arange(121)
ls_ego_vx_count = np.zeros(121)
for i in range(len(ls_ego_vx)):
    k = int(ls_ego_vx[i])
    ls_ego_vx_count[k] += 1
plt.title('ego_vx')
plt.bar(ls_ego_vx_cat, ls_ego_vx_count, width = 0.7, color='blue')
plt.show()

In [ ]:
min(ls_yaw_rate), max(ls_yaw_rate)

In [ ]:
# check the yaw_rate distribution
# -31 24 -> 56
ls_yaw_rate_cat = np.arange(56)
ls_yaw_rate_count = np.zeros(56)
for i in range(len(ls_yaw_rate)):
    k = int(ls_yaw_rate[i])
    ls_yaw_rate_count[k] += 1
for i in range(len(ls_yaw_rate_cat)):
    ls_yaw_rate_cat[i] = ls_yaw_rate_cat[i]-31
plt.title('yaw_rate')
plt.bar(ls_yaw_rate_cat, ls_yaw_rate_count, width = 0.7, color='blue')
plt.show()

In [ ]:
min(ls_sas_angle), max(ls_sas_angle)

In [ ]:
min(ls_long_acc), max(ls_long_acc)

In [ ]:
min(ls_lat_acc), max(ls_lat_acc)

#### Loss function

In [10]:
# get lateral point from the polynomial equation
# input 
# : longitudinal distance, C1, C2, C3
# output 
# : lateral point x

def get_x_polynomial(long_dist, C1, C2, C3):
    mean = 0.0
    inv_sigma = 1.0
    y_ = (long_dist-mean) * inv_sigma
    
    x = C1 * (y_**1) + C2 * (y_**2) + C3 * (y_**3)
    
    return x

In [11]:
# MSELoss from lateral point
# input
# : [C1, C2, C3], gt['ego_global_x', 'ego_global_y']
# output
# : sum, variance, standard deviation of MSELoss

def lp_loss(output, gt):
    criterion = nn.MSELoss()
    # criterion = nn.CrossEntropyLoss()
    mse_total = []
    mse_mean, mse_var, mse_std = 0.0, 0.0, 0.0
    for i in range(len(gt[0][1])):
        x_tmp = get_x_polynomial(gt[0][1][i],output[0][0],output[0][1],output[0][2])
        # make to tensor if needed
        x_tmp_torch = x_tmp
        gt_tmp_torch = gt[0][0][i]
        
        mse_tmp = criterion(x_tmp_torch,gt_tmp_torch)
        mse_total.append(mse_tmp)
    mse_mean = torch.mean(torch.tensor(mse_total))
    mse_var = torch.var(torch.tensor(mse_total))
    mse_std = torch.std(torch.tensor(mse_total))
    
    return mse_mean, mse_var, mse_std

In [ ]:
# Exercise

#### DPENet

In [12]:
# DPENet
# input
# : CAN information (ego_vx, yaw_rate, sas_angle, long_acc, lat_acc)
# output
# : variable for polynomial equation (C1, C2, C3)

# construct model on cuda if available
use_cuda = torch.cuda.is_available()

class DPENet(nn.Module):
    def __init__(self):
        super(DPENet,self).__init__()
        # fullyconncted layer
        fc1 = nn.Linear(5,64,bias=True)
        bn1 = nn.BatchNorm1d(64)
        fc2 = nn.Linear(64,128,bias=True)
        bn2 = nn.BatchNorm1d(128)
        fc3 = nn.Linear(128,256,bias=True)
        bn3 = nn.BatchNorm1d(256)
        fc4 = nn.Linear(256,512,bias=True)
        bn4 = nn.BatchNorm1d(512)
        fc5 = nn.Linear(512,1024,bias=True)
        bn5 = nn.BatchNorm1d(1024)
        fc6 = nn.Linear(1024,1024,bias=True)
        bn6 = nn.BatchNorm1d(1024)
        fc7 = nn.Linear(1024,512,bias=True)
        bn7 = nn.BatchNorm1d(512)
        fc8 = nn.Linear(512,256,bias=True)
        bn8 = nn.BatchNorm1d(256)
        fc9 = nn.Linear(256,128,bias=True)
        bn9 = nn.BatchNorm1d(128)
        fc10 = nn.Linear(128,64,bias=True)
        bn10 = nn.BatchNorm1d(64)
        fc11 = nn.Linear(64,3,bias=True)
        bn11 = nn.BatchNorm1d(3)
        tanh = nn.Tanh()
        dropout = nn.Dropout(0.5)
        #self.relu = nn.ReLU()
        
        # order should be
        # convolution -> batch normalization -> activation -> dropout
        # bn이 dropout을 대체할 수 있다는 의견도 있으나 대체로 같이 사용하였을 때 더 좋은 성능 가져옴
        self.fc_module = nn.Sequential(
            fc1,
            bn1,
            fc2,
            bn2,
            fc3,
            bn3,
            fc4,
            bn4,
            fc5,
            bn5,
            fc6,
            bn6,
            fc7,
            bn7,
            fc8,
            bn8,
            fc9,
            bn9,
            fc10,
            bn10,
            tanh,
            dropout,
            fc11
        )
        # use gpu
        if use_cuda:
            self_fc_module = self.fc_module.cuda()
        
    def forward(self,x):
        out = self.fc_module(x)
        
        return out

In [13]:
print(DPENet())

DPENet(
  (fc_module): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=128, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=256, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=1024, out_features=1024, bias=True)
    (11): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=1024, out_features=512

In [ ]:
# Exercise

#### Train

In [14]:
# Xavier Normal initialization
def weight_init(m): 
    if isinstance(m, nn.Linear):
        size = m.weight.size()
        fan_out = size[0] # number of rows
        fan_in = size[1] # number of columns
        variance = np.sqrt(2.0/(fan_in + fan_out))
        m.weight.data.normal_(0.0, variance)

In [15]:
# argument
# learning rate and beta1,2 for optimizer
dpenet_learning_rate = 0.001 # 0.01(x) 0.001 0.0002
dpenet_momentum = 0.9
dpenet_beta1 = 0.5
dpenet_beta2 = 0.999
# lambda for loss
lambda_mean = 10**-7
lambda_var = 10**-10
lambda_std = 10**-7
# number of epoch and log steo
num_epochs = 1000
log_step = 100
dpenet_batch_size = 64

In [ ]:
# dataloader
data_loader = data.DataLoader(dataset=dpe_dataset, batch_size=dpenet_batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=dpe_testset, batch_size=dpenet_batch_size)

# tensorboardX summary
summary_writer = SummaryWriter('log')
dpenet = DPENet()
dpenet.apply(weight_init)

# To make graph in summary
dummmy = torch.zeros(dpenet_batch_size,5)
summary_writer.add_graph(dpenet,dummmy.cuda())

# Optimizer
#dpenet_optimizer = optim.Adam(dpenet.parameters(), lr=dpenet_learning_rate)
#dpenet_optimizer = optim.Adam(dpenet.parameters(), dpenet_learning_rate, [dpenet_beta1, dpenet_beta2])
dpenet_optimizer = optim.SGD(dpenet.parameters(), lr=dpenet_learning_rate, momentum=dpenet_momentum)

for epoch in range(num_epochs):
    loss_train = 0
    loss_train_lsit = []
    for i, sample in enumerate(data_loader):
        
        # input : ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']
        # gt : ['ego_global_x', 'ego_global_y']
        input_ = sample['in'].cuda()
        gt_ = sample['gt']
        output_ = dpenet(input_)
        #print(output_)
        # train dpenet
        # reset the gradient for the optimizer
        # zero_grad : 역전파 실행 전 변화도 0으로
        dpenet_optimizer.zero_grad()
        
        # loss from lateral position
        loss_lp_mean, loss_lp_var, loss_lp_std = lp_loss(output_,gt_)
        loss_lp_mean = lambda_mean*loss_lp_mean
        loss_lp_var = lambda_var*loss_lp_var
        loss_lp_std = lambda_std*loss_lp_std
        loss = loss_lp_mean + loss_lp_var + loss_lp_std
        loss = Variable(loss_lp_mean,requires_grad=True)
        # backpropagate the gradient
        loss.backward()
        # update the weights using the gradient with the optimizer
        dpenet_optimizer.step()
        
        loss_train_lsit.append(loss/len(input_))
        # print the log information
        if (i+1) % log_step == 0:
            print('Epoch [%d/%d], BatchStep[%d/%d]' % (epoch + 1, num_epochs, i + 1, len(data_loader)))
            print('Loss: %.4f, lp_mean: %.4f, lp_var: %.4f, lp_std: %.4f' 
                  % (loss.data, loss_lp_mean.data, loss_lp_var.data, loss_lp_std.data))

    loss_test = 0
    loss_test_lsit = []
    with torch.no_grad():
        for i, sample_t in enumerate(test_loader):
            input_t = sample_t['in'].cuda()
            gt_t = sample_t['gt']
            output_t = dpenet(input_t)
            loss_lp_mean_t, loss_lp_var_t, loss_lp_std_t = lp_loss(output_t,gt_t)
            loss_test_lsit.append(loss_lp_mean_t/len(input_t))
            
    loss_test = torch.mean(torch.tensor(loss_test_lsit,requires_grad=False))
    loss_train = torch.mean(torch.tensor(loss_train_lsit,requires_grad=False))
    
    print('Epoch [%d/%d]' % (epoch + 1, num_epochs))
    print('Train Loss : %4f, Test Loss: %.4f' % (loss_train.data, loss_test.data))
    
    # visualize the parameter
    for name, param in dpenet.named_parameters():
        #summary.add_histogram(name, param, epoch)
        summary_writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch, bins='doane')
    
    # save summary
    summary_writer.add_scalar('loss/train', loss_train.item(), epoch)
    #summary_writer.add_scalar('loss/loss_lp_mean', loss_lp_mean.item(), epoch)
    #summary_writer.add_scalar('loss/loss_lp_var', loss_lp_var.item(), epoch)
    #summary_writer.add_scalar('loss/loss_lp_std', loss_lp_std.item(), epoch)
    summary_writer.add_scalar('loss/test', loss_test.item(), epoch)
    
    summary_writer.flush()
    # save the model parameters
    #if epoch % 10 == 0:
    model_path = os.path.join('./DPENet_model', 'DPENet-%d.pkl' % (epoch + 1))
    torch.save(dpenet.state_dict(), model_path)

Epoch [1/1000], BatchStep[100/6642]
Loss: 1465.9196, lp_mean: 1465.9196, lp_var: 61888733184.0000, lp_std: 2487.7446
Epoch [1/1000], BatchStep[200/6642]
Loss: 100.7890, lp_mean: 100.7890, lp_var: 251179344.0000, lp_std: 158.4864
Epoch [1/1000], BatchStep[300/6642]
Loss: 466.2022, lp_mean: 466.2022, lp_var: 6301857792.0000, lp_std: 793.8425
Epoch [1/1000], BatchStep[400/6642]
Loss: 12325.1504, lp_mean: 12325.1504, lp_var: 4308859617280.0000, lp_std: 20757.7930
Epoch [1/1000], BatchStep[500/6642]
Loss: 69462.5000, lp_mean: 69462.5000, lp_var: 136787569672192.0000, lp_std: 116956.2188
Epoch [1/1000], BatchStep[600/6642]
Loss: 6671.3252, lp_mean: 6671.3252, lp_var: 1353330065408.0000, lp_std: 11633.2715
Epoch [1/1000], BatchStep[700/6642]
Loss: 305.5999, lp_mean: 305.5999, lp_var: 2610588416.0000, lp_std: 510.9392
Epoch [1/1000], BatchStep[800/6642]
Loss: 16247.9180, lp_mean: 16247.9180, lp_var: 7321226838016.0000, lp_std: 27057.7676
Epoch [1/1000], BatchStep[900/6642]
Loss: 7122.2446, lp_

Epoch [2/1000], BatchStep[500/6642]
Loss: 1441.8594, lp_mean: 1441.8594, lp_var: 62449348608.0000, lp_std: 2498.9866
Epoch [2/1000], BatchStep[600/6642]
Loss: 2.7045, lp_mean: 2.7045, lp_var: 118127.3359, lp_std: 3.4370
Epoch [2/1000], BatchStep[700/6642]
Loss: 1976.7640, lp_mean: 1976.7640, lp_var: 124046245888.0000, lp_std: 3522.0203
Epoch [2/1000], BatchStep[800/6642]
Loss: 50.0910, lp_mean: 50.0910, lp_var: 66338500.0000, lp_std: 81.4484
Epoch [2/1000], BatchStep[900/6642]
Loss: 129.5639, lp_mean: 129.5639, lp_var: 487714784.0000, lp_std: 220.8427
Epoch [2/1000], BatchStep[1000/6642]
Loss: 42045.3516, lp_mean: 42045.3516, lp_var: 48521575137280.0000, lp_std: 69657.4297
Epoch [2/1000], BatchStep[1100/6642]
Loss: 977.0900, lp_mean: 977.0900, lp_var: 27018045440.0000, lp_std: 1643.7167
Epoch [2/1000], BatchStep[1200/6642]
Loss: 37219.6914, lp_mean: 37219.6914, lp_var: 38902115074048.0000, lp_std: 62371.5625
Epoch [2/1000], BatchStep[1300/6642]
Loss: 126722.8125, lp_mean: 126722.8125, 

Epoch [3/1000], BatchStep[900/6642]
Loss: 3.6515, lp_mean: 3.6515, lp_var: 333594.8750, lp_std: 5.7758
Epoch [3/1000], BatchStep[1000/6642]
Loss: 4921.6445, lp_mean: 4921.6445, lp_var: 676501848064.0000, lp_std: 8224.9736
Epoch [3/1000], BatchStep[1100/6642]
Loss: 6584.3716, lp_mean: 6584.3716, lp_var: 1224556675072.0000, lp_std: 11065.9688
Epoch [3/1000], BatchStep[1200/6642]
Loss: 0.0227, lp_mean: 0.0227, lp_var: 6.3168, lp_std: 0.0251
Epoch [3/1000], BatchStep[1300/6642]
Loss: 5681.3042, lp_mean: 5681.3042, lp_var: 882726600704.0000, lp_std: 9395.3535
Epoch [3/1000], BatchStep[1400/6642]
Loss: 91288.6797, lp_mean: 91288.6797, lp_var: 237485166690304.0000, lp_std: 154105.5312
Epoch [3/1000], BatchStep[1500/6642]
Loss: 89.5338, lp_mean: 89.5338, lp_var: 134629616.0000, lp_std: 116.0300
Epoch [3/1000], BatchStep[1600/6642]
Loss: 31.5613, lp_mean: 31.5613, lp_var: 28346392.0000, lp_std: 53.2413
Epoch [3/1000], BatchStep[1700/6642]
Loss: 144464.0781, lp_mean: 144464.0781, lp_var: 5813036

Epoch [4/1000], BatchStep[1400/6642]
Loss: 62382.5508, lp_mean: 62382.5508, lp_var: 110140460105728.0000, lp_std: 104947.8281
Epoch [4/1000], BatchStep[1500/6642]
Loss: 13.7114, lp_mean: 13.7114, lp_var: 6086109.0000, lp_std: 24.6700
Epoch [4/1000], BatchStep[1600/6642]
Loss: 130821.7734, lp_mean: 130821.7734, lp_var: 480556139675648.0000, lp_std: 219215.9062
Epoch [4/1000], BatchStep[1700/6642]
Loss: 20592.4316, lp_mean: 20592.4316, lp_var: 12012719439872.0000, lp_std: 34659.3711
Epoch [4/1000], BatchStep[1800/6642]
Loss: 8061.3687, lp_mean: 8061.3687, lp_var: 1758078959616.0000, lp_std: 13259.2578
Epoch [4/1000], BatchStep[1900/6642]
Loss: 0.7975, lp_mean: 0.7975, lp_var: 15965.5850, lp_std: 1.2635
Epoch [4/1000], BatchStep[2000/6642]
Loss: 2314.7993, lp_mean: 2314.7993, lp_var: 148081410048.0000, lp_std: 3848.1348
Epoch [4/1000], BatchStep[2100/6642]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [4/1000], BatchStep[2200/6642]
Loss: 61198.7188, lp_mean: 61198.71

Epoch [5/1000], BatchStep[1800/6642]
Loss: 5883.4727, lp_mean: 5883.4727, lp_var: 992461783040.0000, lp_std: 9962.2383
Epoch [5/1000], BatchStep[1900/6642]
Loss: 1972.0753, lp_mean: 1972.0753, lp_var: 118830448640.0000, lp_std: 3447.1792
Epoch [5/1000], BatchStep[2000/6642]
Loss: 4.3939, lp_mean: 4.3939, lp_var: 799987.7500, lp_std: 8.9442
Epoch [5/1000], BatchStep[2100/6642]
Loss: 109005.9219, lp_mean: 109005.9219, lp_var: 340653384925184.0000, lp_std: 184567.9844
Epoch [5/1000], BatchStep[2200/6642]
Loss: 5.9508, lp_mean: 5.9508, lp_var: 856769.8750, lp_std: 9.2562
Epoch [5/1000], BatchStep[2300/6642]
Loss: 6.4438, lp_mean: 6.4438, lp_var: 1236853.2500, lp_std: 11.1214
Epoch [5/1000], BatchStep[2400/6642]
Loss: 2400.1077, lp_mean: 2400.1077, lp_var: 163007021056.0000, lp_std: 4037.4128
Epoch [5/1000], BatchStep[2500/6642]
Loss: 2111.2822, lp_mean: 2111.2822, lp_var: 122043850752.0000, lp_std: 3493.4775
Epoch [5/1000], BatchStep[2600/6642]
Loss: 309.7025, lp_mean: 309.7025, lp_var: 29

...

Epoch [11/1000], BatchStep[4400/6642]
Loss: 1.4615, lp_mean: 1.4615, lp_var: 59341.4570, lp_std: 2.4360
Epoch [11/1000], BatchStep[4500/6642]
Loss: 2406.9097, lp_mean: 2406.9097, lp_var: 168963997696.0000, lp_std: 4110.5234
Epoch [11/1000], BatchStep[4600/6642]
Loss: 8209.0869, lp_mean: 8209.0869, lp_var: 1946061897728.0000, lp_std: 13950.1328
Epoch [11/1000], BatchStep[4700/6642]
Loss: 593.6962, lp_mean: 593.6962, lp_var: 10410595328.0000, lp_std: 1020.3232
Epoch [11/1000], BatchStep[4800/6642]
Loss: 28.5698, lp_mean: 28.5698, lp_var: 23841380.0000, lp_std: 48.8276
Epoch [11/1000], BatchStep[4900/6642]
Loss: 9060.5605, lp_mean: 9060.5605, lp_var: 2087550844928.0000, lp_std: 14448.3604
Epoch [11/1000], BatchStep[5000/6642]
Loss: 13822.2754, lp_mean: 13822.2754, lp_var: 5270864920576.0000, lp_std: 22958.3652
Epoch [11/1000], BatchStep[5100/6642]
Loss: 47788.5625, lp_mean: 47788.5625, lp_var: 64170443669504.0000, lp_std: 80106.4531
Epoch [11/1000], BatchStep[5200/6642]
Loss: 32.0705, lp_

Epoch [12/1000], BatchStep[4700/6642]
Loss: 1.8556, lp_mean: 1.8556, lp_var: 172400.4844, lp_std: 4.1521
Epoch [12/1000], BatchStep[4800/6642]
Loss: 11474.2412, lp_mean: 11474.2412, lp_var: 3688944107520.0000, lp_std: 19206.6250
Epoch [12/1000], BatchStep[4900/6642]
Loss: 149.2718, lp_mean: 149.2718, lp_var: 582052608.0000, lp_std: 241.2577
Epoch [12/1000], BatchStep[5000/6642]
Loss: 3907.5513, lp_mean: 3907.5513, lp_var: 426374004736.0000, lp_std: 6529.7319
Epoch [12/1000], BatchStep[5100/6642]
Loss: 4.7057, lp_mean: 4.7057, lp_var: 378571.6562, lp_std: 6.1528
Epoch [12/1000], BatchStep[5200/6642]
Loss: 1507.0973, lp_mean: 1507.0973, lp_var: 66356432896.0000, lp_std: 2575.9741
Epoch [12/1000], BatchStep[5300/6642]
Loss: 39237.6836, lp_mean: 39237.6836, lp_var: 44544934543360.0000, lp_std: 66741.9922
Epoch [12/1000], BatchStep[5400/6642]
Loss: 0.6677, lp_mean: 0.6677, lp_var: 10360.4600, lp_std: 1.0179
Epoch [12/1000], BatchStep[5500/6642]
Loss: 164540.5625, lp_mean: 164540.5625, lp_va

Epoch [13/1000], BatchStep[5000/6642]
Loss: 673254.8750, lp_mean: 673254.8750, lp_var: 12837631477940224.0000, lp_std: 1133032.7500
Epoch [13/1000], BatchStep[5100/6642]
Loss: 43707.8438, lp_mean: 43707.8438, lp_var: 54448181215232.0000, lp_std: 73789.0078
Epoch [13/1000], BatchStep[5200/6642]
Loss: 0.7083, lp_mean: 0.7083, lp_var: 6094.1431, lp_std: 0.7806
Epoch [13/1000], BatchStep[5300/6642]
Loss: 0.0104, lp_mean: 0.0104, lp_var: 6.4299, lp_std: 0.0254
Epoch [13/1000], BatchStep[5400/6642]
Loss: 971.7686, lp_mean: 971.7686, lp_var: 24708128768.0000, lp_std: 1571.8820
Epoch [13/1000], BatchStep[5500/6642]
Loss: 46106.1680, lp_mean: 46106.1680, lp_var: 59818308009984.0000, lp_std: 77342.2969
Epoch [13/1000], BatchStep[5600/6642]
Loss: 90.8568, lp_mean: 90.8568, lp_var: 248045072.0000, lp_std: 157.4945
Epoch [13/1000], BatchStep[5700/6642]
Loss: 475.3900, lp_mean: 475.3900, lp_var: 5670690816.0000, lp_std: 753.0399
Epoch [13/1000], BatchStep[5800/6642]
Loss: 1170.3671, lp_mean: 1170.36

Epoch [14/1000], BatchStep[5500/6642]
Loss: 5112.5674, lp_mean: 5112.5674, lp_var: 783017050112.0000, lp_std: 8848.8252
Epoch [14/1000], BatchStep[5600/6642]
Loss: 160.7090, lp_mean: 160.7090, lp_var: 725743296.0000, lp_std: 269.3962
Epoch [14/1000], BatchStep[5700/6642]
Loss: 10275.0928, lp_mean: 10275.0928, lp_var: 2901519630336.0000, lp_std: 17033.8477
Epoch [14/1000], BatchStep[5800/6642]
Loss: 9563.0635, lp_mean: 9563.0635, lp_var: 2380090834944.0000, lp_std: 15427.5430
Epoch [14/1000], BatchStep[5900/6642]
Loss: 1958.0146, lp_mean: 1958.0146, lp_var: 112597450752.0000, lp_std: 3355.5544
Epoch [14/1000], BatchStep[6000/6642]
Loss: 366.9285, lp_mean: 366.9285, lp_var: 3832185088.0000, lp_std: 619.0464
Epoch [14/1000], BatchStep[6100/6642]
Loss: 189.4476, lp_mean: 189.4476, lp_var: 791262080.0000, lp_std: 281.2938
Epoch [14/1000], BatchStep[6200/6642]
Loss: 2.5895, lp_mean: 2.5895, lp_var: 168256.5000, lp_std: 4.1019
Epoch [14/1000], BatchStep[6300/6642]
Loss: 33.3402, lp_mean: 33.3

Epoch [15/1000], BatchStep[5900/6642]
Loss: 3335.5015, lp_mean: 3335.5015, lp_var: 319773966336.0000, lp_std: 5654.8560
Epoch [15/1000], BatchStep[6000/6642]
Loss: 166.6924, lp_mean: 166.6924, lp_var: 796579776.0000, lp_std: 282.2375
Epoch [15/1000], BatchStep[6100/6642]
Loss: 7262.3828, lp_mean: 7262.3828, lp_var: 1503607652352.0000, lp_std: 12262.1680
Epoch [15/1000], BatchStep[6200/6642]
Loss: 2.5482, lp_mean: 2.5482, lp_var: 204497.7656, lp_std: 4.5221
Epoch [15/1000], BatchStep[6300/6642]
Loss: 335.4832, lp_mean: 335.4832, lp_var: 3214754048.0000, lp_std: 566.9880
Epoch [15/1000], BatchStep[6400/6642]
Loss: 220117.4688, lp_mean: 220117.4688, lp_var: 1333279315722240.0000, lp_std: 365141.0000
Epoch [15/1000], BatchStep[6500/6642]
Loss: 139488.0469, lp_mean: 139488.0469, lp_var: 551861220278272.0000, lp_std: 234917.2500
Epoch [15/1000], BatchStep[6600/6642]
Loss: 18631.6641, lp_mean: 18631.6641, lp_var: 9727811518464.0000, lp_std: 31189.4375
Epoch [15/1000]
Train Loss : 585.785583, 

In [ ]:
print(gt_.requires_grad)

In [ ]:
print(input_.requires_grad)

In [ ]:
print(output_.requires_grad)

In [ ]:
print(loss_test.requires_grad)

In [ ]:
loss_lp_mean, loss_lp_var, loss_lp_std = lp_loss(output_,gt_)

In [ ]:
print(loss_lp_mean.requires_grad)

In [ ]:
loss = Variable(loss_lp_mean,requires_grad=True)

In [ ]:
print(loss.requires_grad)

In [ ]:
loss.backward()

In [ ]:
for name, param in dpenet.named_parameters():
    if param.grad is not None:
        print(name, param.grad.sum())
    else:
        print(name, param.grad)

In [ ]:
for name, param in dpenet.named_parameters():
    print(name, param.requires_grad)

In [ ]:
print(loss_test/len(data_loader))

In [ ]:
len(input_)

In [ ]:
loss.retain_grad()
loss.backward()

In [ ]:
print(loss.grad)

In [ ]:
# Exercise

In [ ]:
len(data_loader)


In [ ]:
# w_real_deep


In [ ]:
# w_real_deep_wo_decoder
Epoch [1/1000] Test Loss: 144666329088.0000
Epoch [2/1000] Test Loss: 145073176576.0000
Epoch [3/1000] Test Loss: 146446860288.0000
Epoch [4/1000] Test Loss: 144554934272.0000
Epoch [5/1000] Test Loss: 145652776960.0000
Epoch [6/1000] Test Loss: 143921414144.0000

In [ ]:
# w_dropout_wo_loss_scaling
epoch [1/1000] Test Loss: 236042076160.0000
epoch [2/1000] Test Loss: 238885634048.0000 
Epoch [3/1000] Test Loss: 235835916288.0000
Epoch [4/1000] Test Loss: 237683245056.0000
Epoch [5/1000] Test Loss: 240484368384.0000
Epoch [6/1000] Test Loss: 235287920640.0000
Epoch [7/1000] Test Loss: 232444411904.0000
Epoch [8/1000] Test Loss: 233658007552.0000
Epoch [9/1000] Test Loss: 235095965696.0000
Epoch [10/1000] Test Loss: 232347713536.0000
Epoch [11/1000] Test Loss: 239848275968.0000
Epoch [12/1000] Test Loss: 242579668992.0000
Epoch [13/1000] Test Loss: 240169140224.0000
Epoch [14/1000] Test Loss: 239437922304.0000
